# Install Libraries

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

In [2]:
!pip install finrl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.2/127.2 kB 3.1 MB/s eta 0:00:00


In [3]:
!pip install stable_baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [4]:
!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

  Cloning https://github.com/AI4Finance-LLC/FinRL-Library.git to /tmp/pip-req-build-qy3gslv1
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-LLC/FinRL-Library.git /tmp/pip-req-build-qy3gslv1
  Resolved https://github.com/AI4Finance-LLC/FinRL-Library.git to commit d25d902a6de54931a329adc38a2663e8f576adc4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-pd4drii9/elegantrl_deabdf87bd694376b3641bba5abaf11b
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-pd4drii9/elegantrl_deabdf87bd694376b3641bba5abaf11b
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit 5e828af1503098f4da046c0f12432dbd4ef8bd97
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.

In [5]:
!pip install pandas_market_calendars

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.0/123.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.4/198.4 kB 10.1 MB/s eta 0:00:00


In [6]:
!git clone https://github.com/benstaf/FinRL_DeepSeek.git

Cloning into 'FinRL_DeepSeek'...
remote: Enumerating objects: 145, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (144/144), done.
remote: Total 145 (delta 94), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (145/145), 1.21 MiB | 7.31 MiB/s, done.
Resolving deltas: 100% (94/94), done.


In [7]:
!git clone https://github.com/benstaf/spinningup_pytorch.git

Cloning into 'spinningup_pytorch'...
remote: Enumerating objects: 1299, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 1299 (delta 13), reused 7 (delta 7), pack-reused 1279 (from 3)
Receiving objects: 100% (1299/1299), 31.03 MiB | 16.96 MiB/s, done.
Resolving deltas: 100% (609/609), done.


In [8]:
!pip install torch torchvision torchaudio
!pip install numpy scipy gymnasium pandas matplotlib seaborn
!pip install mpi4py tqdm cloudpickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.3/466.3 kB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-4.0.3-cp311-cp311-linux_x86_64.whl size=4458266 sha256=7195e9b987af8106a6928c3abb7ac50653b0d7b171d516a34ad57b71999c3bff
  Stored in directory: /root/.cache/pip/wheels/5c/56/17/bf6ba37aa971a191a8b9eaa188bf5ec855b8911c1c56fb1f84
Successfully built mpi4py


In [9]:
!apt-get update
!apt-get install -y swig cmake build-essential
!pip install --upgrade pip setuptools wheel

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [73.0 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,836 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,101 kB]
Ge

In [1]:
!pip install box2d box2d-py

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 49.1 MB/s eta 0:00:00
  Created wheel for box2d-py: filename=box2d_py-2.3.8-cp311-cp311-linux_x86_64.whl size=2351249 sha256=baf6e7bac065c4142820484aa79a808e6ef9357ae19f553e8bd8f1d32ae674bf
  Stored in directory: /root/.cache/pip/wheels/3f/95/02/4cb5adc9f6dcaeb9639c2271f630a66ab4440102414804c45c
Successfully built box2d-py


In [2]:
!pip install "gym==0.15.7"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.3 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.15.7-py3-none-any.whl size=1648866 sha256=c5416d991586a14b09dc302114257e6377a6b20fd13afcc670cc8cb3ccf9d6f4
  Stored in directory: /root/.cache/pip/wheels/73/0f/f8/6d956af0cb6d2b9daf2ac52995d2be56dccddd8c0c3f9e6454
Successfully built gym
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 3.1.1
    Uninstalling cloudpickle-3.1.1:
      Successfully uninstalled cloudpickle-3.1.1
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.42.0 requires cloudpickl

In [3]:
!pip install -e .

Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


# Import Libraries


In [4]:
%cd /content/spinningup_pytorch

/content/spinningup_pytorch


In [5]:
import sys
sys.path.append("/content/FinRL_DeepSeek/")  # Adjust path to root folder
from finrl.config import INDICATORS, TRAINED_MODEL_DIR, RESULTS_DIR
#from finrl.main import check_and_make_directories
#from env_stocktrading import StockTradingEnv
import os

def check_and_make_directories(directories):
    for directory in directories:
        os.makedirs("./" + directory, exist_ok=True)  # This prevents FileEx>


check_and_make_directories([TRAINED_MODEL_DIR])
sys.path.append("/content/spinningup_pytorch")

In [6]:
#!/usr/bin/env python
# coding: utf-8

from datasets import load_dataset
import pandas as pd
#from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.config import INDICATORS, TRAINED_MODEL_DIR, RESULTS_DIR
from finrl.main import check_and_make_directories
from env_stocktrading_llm_01 import StockTradingEnv

check_and_make_directories([TRAINED_MODEL_DIR])


import numpy as np
import scipy.signal
from gymnasium.spaces import Box, Discrete

import torch
import torch.nn as nn
from torch.distributions.normal import Normal
from torch.distributions.categorical import Categorical


###############

In [7]:
import numpy as np
import torch
from torch.optim import Adam
import gymnasium as gym
import time
import spinup.algos.pytorch.ppo.core as core
from spinup.utils.logx import EpochLogger
from spinup.utils.mpi_pytorch import setup_pytorch_for_mpi, sync_params, mpi_avg_grads
from spinup.utils.mpi_tools import mpi_fork, mpi_avg, proc_id, mpi_statistics_scalar, num_procs


# Code

In [8]:
#train = pd.read_csv('train_data2.csv')
#dataset = load_dataset("benstaf/train_data_qwen_sentiment", data_files="train_data_qwen_sentiment.csv")
dataset = load_dataset("benstaf/nasdaq_2013_2023", data_files="train_data_deepseek_sentiment_2013_2018.csv")

# Convert to pandas DataFrame
train = pd.DataFrame(dataset['train'])

#train = pd.read_csv('train_data_qwen_sentiment.csv')

train = train.drop('Unnamed: 0',axis=1)

# Create a new index based on unique dates
unique_dates = train['date'].unique()
date_to_idx = {date: idx for idx, date in enumerate(unique_dates)}

# Create new index based on the date mapping
train['new_idx'] = train['date'].map(date_to_idx)

# Set this as the index
train = train.set_index('new_idx')


#missing values with 0
train['llm_sentiment'].fillna(0, inplace=True)


# If you are not using the data generated from part 1 of this tutorial, make sure
# it has the columns and index in the form that could be make into the environment.
# Then you can comment and skip the following two lines.
#train = train.set_index(train.columns[0])
#train.index.names = ['']


# ## Construct the environment

# Calculate and specify the parameters we need for constructing the environment.

# In[16]:


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


(…)in_data_deepseek_sentiment_2013_2018.csv:   0%|          | 0.00/37.0M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

<ipython-input-8-9a1ecc3c6c86>:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['llm_sentiment'].fillna(0, inplace=True)


In [9]:
train

,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence,llm_sentiment
new_idx,,,,,,,,,,,,,,,,,,,
0,2013-01-02,AAPL,16.687340,19.821428,19.343929,19.779285,560518000.0,2.0,0.000000,16.879896,16.284159,0.000000,-66.666667,100.000000,16.687340,16.687340,14.68,0.000000,0.0
0,2013-01-02,ADBE,38.340000,38.730000,37.919998,37.919998,6483800.0,2.0,0.000000,16.879896,16.284159,0.000000,-66.666667,100.000000,38.340000,38.340000,14.68,0.000000,0.0
0,2013-01-02,ADI,33.714191,44.009998,42.799999,42.799999,3761100.0,2.0,0.000000,16.879896,16.284159,0.000000,-66.666667,100.000000,33.714191,33.714191,14.68,0.000000,3.0
0,2013-01-02,ADP,39.267780,51.404739,50.702370,50.974541,3217561.0,2.0,0.000000,16.879896,16.284159,0.000000,-66.666667,100.000000,39.267780,39.267780,14.68,0.000000,0.0
0,2013-01-02,ADSK,36.939999,36.950001,36.049999,36.049999,2998800.0,2.0,0.000000,16.879896,16.284159,0.000000,-66.666667,100.000000,36.939999,36.939999,14.68,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1508,2018-12-28,VRTX,161.419998,164.839996,160.380005,163.520004,966700.0,4.0,-3.772674,187.585850,150.832147,44.935760,-57.015038,17.249474,168.107999,172.781499,28.34,53.769986,0.0
1508,2018-12-28,WBA,50.158054,69.080002,67.589996,68.559998,5181800.0,4.0,-2.631320,67.057912,46.443012,37.786170,-140.581480,57.411928,57.942806,57.250959,28.34,53.769986,3.0
1508,2018-12-28,WBD,24.639999,25.049999,24.400000,24.850000,3404300.0,4.0,-1.549457,31.365200,23.523799,36.637987,-143.645140,51.485596,28.545333,30.469833,28.34,53.769986,0.0


In [10]:
train.dtypes.reset_index().rename(columns={"index": "Column", 0: "Data Type"})


,Column,Data Type
0,date,object
1,tic,object
2,close,float64
3,high,float64
4,low,float64
5,open,float64
6,volume,float64
7,day,float64
8,macd,float64
9,boll_ub,float64


In [11]:
print(train['date'].dtype)


object


In [12]:
print(train['date'][0].dtype)


object


In [13]:
print(train['tic'][0])


new_idx
0    AAPL
0    ADBE
0     ADI
0     ADP
0    ADSK
     ... 
0    VRTX
0     WBA
0     WBD
0    WDAY
0     XEL
Name: tic, Length: 84, dtype: object


In [14]:
print(type(train['llm_sentiment']))

<class 'pandas.core.series.Series'>


In [15]:
# List of desired tickers
# tickers = ['AMD', 'ADBE', 'PYPL', 'CSCO', 'NFLX', 'NVDA', 'EBAY', 'MNST', 'GOOGL']

# # Filter and sort
# train = train[train['tic'].isin(tickers)].sort_values(by='date')

# Ticker list


# Step 2: Filter by tickers and date range
tickers = ['AMD', 'ADBE', 'PYPL', 'CSCO', 'NFLX', 'NVDA', 'EBAY', 'MNST', 'GOOGL']
train = train[
    (train['tic'].isin(tickers)) &
    (train['date'] >= '2018-06-14') &
    (train['date'] <= '2018-12-28')
].sort_values(by='date')

# Step 3: Create a mapping from unique dates to a new index
date_to_idx = {date: idx for idx, date in enumerate(sorted(train['date'].unique()))}

# Step 4: Map that to a new column
train['new_idx'] = train['date'].map(date_to_idx)

# Step 5: Set new_idx as the actual DataFrame index
train.set_index('new_idx', inplace=True)

In [16]:
# Drop the 'new_idx' column (not the index!)
if 'new_idx' in train.columns:
    train.drop(columns='new_idx', inplace=True)

In [17]:
train.tail(100)


,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence,llm_sentiment
new_idx,,,,,,,,,,,,,,,,,,,
124,2018-12-11,GOOGL,52.890766,53.520000,52.505001,53.347000,33852000.0,1.0,-0.292657,55.421553,50.420711,46.146499,-18.970821,10.790669,53.111915,55.169663,21.760000,72.383614,3.0
124,2018-12-11,MNST,28.790001,29.174999,28.625000,28.780001,7694600.0,1.0,0.384261,30.129869,26.993131,52.101490,63.865939,8.540937,28.075667,27.941333,21.760000,72.383614,0.0
124,2018-12-11,NFLX,265.320007,274.500000,262.760010,274.079987,9843200.0,1.0,-10.289980,301.423549,253.002455,41.331594,-88.093975,25.572756,287.278002,319.073834,21.760000,72.383614,3.0
124,2018-12-11,NVDA,3.674931,3.897250,3.625000,3.889000,671912000.0,1.0,-0.370519,4.975526,3.096695,36.694613,-75.760034,39.340276,4.409031,5.349694,21.760000,72.383614,3.0
125,2018-12-12,NFLX,274.880005,281.769989,266.480011,267.660004,11456700.0,2.0,-9.586523,299.135749,253.371255,43.554691,-54.446088,16.726182,286.946002,317.816000,21.459999,63.311663,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,2018-12-28,CSCO,35.352314,43.529999,42.459999,43.169998,22447300.0,4.0,-0.933148,41.644856,33.156245,44.278809,-94.225460,24.601468,37.500291,37.762864,28.340000,53.769986,0.0
136,2018-12-28,EBAY,25.516727,28.639999,28.020000,28.110001,8781000.0,4.0,-0.393859,27.661340,24.065148,44.415219,-29.248687,3.424477,25.807373,26.498000,28.340000,53.769986,3.0
136,2018-12-28,GOOGL,52.144970,53.211498,52.099998,52.974998,34398000.0,4.0,-0.585858,55.819963,49.164361,46.819479,-8.761724,2.705845,52.473596,53.664628,28.340000,53.769986,3.0


In [18]:
train.shape

(1096, 19)

In [19]:
train.to_csv('/content/working_train_ppo_llm_their_dataset.csv')

In [20]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + (1+ len(INDICATORS))*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 8, State Space: 89


In [21]:
type(INDICATORS)

list

In [22]:
train.head()

,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence,llm_sentiment
new_idx,,,,,,,,,,,,,,,,,,,
0,2018-06-14,ADBE,258.100006,258.910004,256.579987,257.149994,5618300.0,3.0,5.850034,260.433595,234.682404,65.660710,138.798105,29.521975,243.546666,232.777001,12.12,89.060903,3.0
0,2018-06-14,AMD,16.250000,16.790001,15.580000,16.620001,113048600.0,3.0,1.106576,16.752204,11.928796,71.102785,143.016757,65.419166,13.525000,11.878500,12.12,89.060903,4.0
0,2018-06-14,CSCO,36.407501,44.810001,44.049999,44.230000,24020500.0,3.0,-0.010623,36.257351,34.760486,54.407913,23.549965,12.911593,36.049857,35.559181,12.12,89.060903,3.0
0,2018-06-14,EBAY,35.239101,39.669998,38.950001,39.520000,9353400.0,3.0,0.255263,37.035818,33.176740,48.326901,53.915311,9.269947,34.890025,35.524782,12.12,89.060903,0.0
0,2018-06-14,GOOGL,57.795986,58.299500,57.529999,57.610500,35422000.0,3.0,1.073167,58.808446,52.078679,59.484620,129.788651,28.994979,54.845397,53.314539,12.12,89.060903,0.0


In [23]:

# In[16]:


buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)


# ## Environment for training

# In[17]:


env_train, _ = e_train_gym.get_sb_env()
#print(type(env_train))


# Initialize the environment
#env_train, _ = e_train_gym.get_sb_env()

# Check the observation space
print("Observation space:", env_train.observation_space)
print("Observation space shape:", env_train.observation_space.shape)

# Reset the environment and check the observation returned
obs = env_train.reset()
print("Observation returned from reset:", obs.shape)



# # Part 3: Train DRL Agents
# * Here, the DRL algorithms are from **[Stable Baselines 3](https://stable-baselines3.readthedocs.io/en/master/)**. It's a library that implemented popular DRL algorithms using pytorch, succeeding to its old version: Stable Baselines.
# * Users are also encouraged to try **[ElegantRL](https://github.com/AI4Finance-Foundation/ElegantRL)** and **[Ray RLlib](https://github.com/ray-project/ray)**.

# In[18]:

#Custom PPO agent


Observation space: Box(-inf, inf, (89,), float32)
Observation space shape: (89,)
Observation returned from reset: (1, 89)


In [24]:
def combined_shape(length, shape=None):
    if shape is None:
        return (length,)
    return (length, shape) if np.isscalar(shape) else (length, *shape)


def mlp(sizes, activation, output_activation=nn.Identity):
    layers = []
    for j in range(len(sizes)-1):
        act = activation if j < len(sizes)-2 else output_activation
        layers += [nn.Linear(sizes[j], sizes[j+1]), act()]
    return nn.Sequential(*layers)


def count_vars(module):
    return sum([np.prod(p.shape) for p in module.parameters()])


def discount_cumsum(x, discount):
    """
    magic from rllab for computing discounted cumulative sums of vectors.

    input:
        vector x,
        [x0,
         x1,
         x2]

    output:
        [x0 + discount * x1 + discount^2 * x2,
         x1 + discount * x2,
         x2]
    """
    return scipy.signal.lfilter([1], [1, float(-discount)], x[::-1], axis=0)[::-1]


In [25]:
class Actor(nn.Module):

    def _distribution(self, obs):
        raise NotImplementedError

    def _log_prob_from_distribution(self, pi, act):
        raise NotImplementedError

    def forward(self, obs, act=None):
        # Produce action distributions for given observations, and
        # optionally compute the log likelihood of given actions under
        # those distributions.
        pi = self._distribution(obs)
        logp_a = None
        if act is not None:
            logp_a = self._log_prob_from_distribution(pi, act)
        return pi, logp_a

In [26]:

class MLPCategoricalActor(Actor):

    def __init__(self, obs_dim, act_dim, hidden_sizes, activation):
        super().__init__()
        self.logits_net = mlp([obs_dim] + list(hidden_sizes) + [act_dim], activation)

    def _distribution(self, obs):
        logits = self.logits_net(obs)
        return Categorical(logits=logits)

    def _log_prob_from_distribution(self, pi, act):
        return pi.log_prob(act)

In [27]:
class MLPGaussianActor(Actor):

    def __init__(self, obs_dim, act_dim, hidden_sizes, activation):
        super().__init__()
        log_std = -0.5 * np.ones(act_dim, dtype=np.float32)
        self.log_std = torch.nn.Parameter(torch.as_tensor(log_std))
        self.mu_net = mlp([obs_dim] + list(hidden_sizes) + [act_dim], activation)

    def _distribution(self, obs):
        mu = self.mu_net(obs)
        std = torch.exp(self.log_std)
        return Normal(mu, std)

    def _log_prob_from_distribution(self, pi, act):
        return pi.log_prob(act).sum(axis=-1)    # Last axis sum needed for Torch Normal distribution

In [28]:
class MLPCritic(nn.Module):

    def __init__(self, obs_dim, hidden_sizes, activation):
        super().__init__()
        self.v_net = mlp([obs_dim] + list(hidden_sizes) + [1], activation)

    def forward(self, obs):
        return torch.squeeze(self.v_net(obs), -1) # Critical to ensure v has right shape.

In [29]:

class MLPActorCritic(nn.Module):
    def __init__(self, observation_space, action_space,
                 hidden_sizes=(64, 64), activation=nn.Tanh):
        super().__init__()

        obs_dim = observation_space.shape[0]

        # policy builder depends on action space
        if isinstance(action_space, Box):
            self.pi = MLPGaussianActor(obs_dim, action_space.shape[0], hidden_sizes, activation)
        elif isinstance(action_space, Discrete):
            self.pi = MLPCategoricalActor(obs_dim, action_space.n, hidden_sizes, activation)

        # build value function
        self.v = MLPCritic(obs_dim, hidden_sizes, activation)

    def step(self, obs):
        with torch.no_grad():
            pi = self.pi._distribution(obs)
            a = pi.sample()
            logp_a = self.pi._log_prob_from_distribution(pi, a)
            v = self.v(obs)
        return a.numpy(), v.numpy(), logp_a.numpy()

    def act(self, obs):
        return self.step(obs)[0]


###############################

In [30]:
class PPOBuffer:
    """
    A buffer for storing trajectories experienced by a PPO agent interacting
    with the environment, and using Generalized Advantage Estimation (GAE-Lambda)
    for calculating the advantages of state-action pairs.
    """

    def __init__(self, obs_dim, act_dim, size, gamma=0.99, lam=0.95):
        self.obs_buf = np.zeros(core.combined_shape(size, obs_dim), dtype=np.float32)
        self.act_buf = np.zeros(core.combined_shape(size, act_dim), dtype=np.float32)
        self.adv_buf = np.zeros(size, dtype=np.float32)
        self.rew_buf = np.zeros(size, dtype=np.float32)
        self.ret_buf = np.zeros(size, dtype=np.float32)
        self.val_buf = np.zeros(size, dtype=np.float32)
        self.logp_buf = np.zeros(size, dtype=np.float32)
        self.gamma, self.lam = gamma, lam
        self.ptr, self.path_start_idx, self.max_size = 0, 0, size

    def store(self, obs, act, rew, val, logp):
        """
        Append one timestep of agent-environment interaction to the buffer.
        """
        assert self.ptr < self.max_size     # buffer has to have room so you can store
        self.obs_buf[self.ptr] = obs
        self.act_buf[self.ptr] = act
        self.rew_buf[self.ptr] = rew.item()
        self.val_buf[self.ptr] = val.item()
        self.logp_buf[self.ptr] = logp.item()
        self.ptr += 1

    def finish_path(self, last_val=0):
        """
        Call this at the end of a trajectory, or when one gets cut off
        by an epoch ending. This looks back in the buffer to where the
        trajectory started, and uses rewards and value estimates from
        the whole trajectory to compute advantage estimates with GAE-Lambda,
        as well as compute the rewards-to-go for each state, to use as
        the targets for the value function.

        The "last_val" argument should be 0 if the trajectory ended
        because the agent reached a terminal state (died), and otherwise
        should be V(s_T), the value function estimated for the last state.
        This allows us to bootstrap the reward-to-go calculation to account
        for timesteps beyond the arbitrary episode horizon (or epoch cutoff).
        """

        path_slice = slice(self.path_start_idx, self.ptr)
        rews = np.append(self.rew_buf[path_slice], last_val)
        vals = np.append(self.val_buf[path_slice], last_val)

        # the next two lines implement GAE-Lambda advantage calculation
        deltas = rews[:-1] + self.gamma * vals[1:] - vals[:-1]
        self.adv_buf[path_slice] = core.discount_cumsum(deltas, self.gamma * self.lam)

        # the next line computes rewards-to-go, to be targets for the value function
        self.ret_buf[path_slice] = core.discount_cumsum(rews, self.gamma)[:-1]

        self.path_start_idx = self.ptr

    def get(self):
        """
        Call this at the end of an epoch to get all of the data from
        the buffer, with advantages appropriately normalized (shifted to have
        mean zero and std one). Also, resets some pointers in the buffer.
        """
        assert self.ptr == self.max_size    # buffer has to be full before you can get
        self.ptr, self.path_start_idx = 0, 0
        # the next two lines implement the advantage normalization trick
        adv_mean, adv_std = mpi_statistics_scalar(self.adv_buf)
        self.adv_buf = (self.adv_buf - adv_mean) / adv_std
        data = dict(obs=self.obs_buf, act=self.act_buf, ret=self.ret_buf,
                    adv=self.adv_buf, logp=self.logp_buf)
        return {k: torch.as_tensor(v, dtype=torch.float32) for k,v in data.items()}


#End definition class PPOBuffer

In [31]:


def ppo(
    env_fn,
    actor_critic=MLPActorCritic,
    ac_kwargs=dict(hidden_sizes=[256, 128], activation=torch.nn.ReLU),
    seed=42,
    steps_per_epoch=8000,  # Larger batch size for better gradient estimation
    epochs=50,  # More epochs for convergence in a volatile environment
    gamma=0.995,  # Higher discount factor to account for long-term rewards
    clip_ratio=0.7,  # it's 90% clipping when Reduced clip ratio for stable updates
    pi_lr=3e-5,  # Lower policy learning rate
    vf_lr=1e-4,  # Lower value function learning rate
    train_pi_iters=100,  # Increased policy training iterations
    train_v_iters=100,  # Increased value function training iterations
    lam=0.95,  # GAE smoothing factor for advantage estimation
    max_ep_len=1000,  # Typical trading day or customizable period
    target_kl=0.35,  # relaxed KL divergence limit
    logger_kwargs=dict(),
    save_freq=5  # Save checkpoints more frequently
):

#OLD PPO hyperparameters:

#def ppo(env_fn, actor_critic=MLPActorCritic, ac_kwargs=dict(), seed=0,
 #       steps_per_epoch=4000, epochs=50, gamma=0.99, clip_ratio=0.2, pi_lr=3e-4,
  #      vf_lr=1e-3, train_pi_iters=80, train_v_iters=80, lam=0.97, max_ep_len=1000,
   #     target_kl=0.01, logger_kwargs=dict(), save_freq=10):
    """
    Proximal Policy Optimization (by clipping),

    with early stopping based on approximate KL

    Args:
        env_fn : A function which creates a copy of the environment.
            The environment must satisfy the OpenAI Gym API.

        actor_critic: The constructor method for a PyTorch Module with a
            ``step`` method, an ``act`` method, a ``pi`` module, and a ``v``
            module. The ``step`` method should accept a batch of observations
            and return:

            ===========  ================  ======================================
            Symbol       Shape             Description
            ===========  ================  ======================================
            ``a``        (batch, act_dim)  | Numpy array of actions for each
                                           | observation.
            ``v``        (batch,)          | Numpy array of value estimates
                                           | for the provided observations.
            ``logp_a``   (batch,)          | Numpy array of log probs for the
                                           | actions in ``a``.
            ===========  ================  ======================================

            The ``act`` method behaves the same as ``step`` but only returns ``a``.

            The ``pi`` module's forward call should accept a batch of
            observations and optionally a batch of actions, and return:

            ===========  ================  ======================================
            Symbol       Shape             Description
            ===========  ================  ======================================
            ``pi``       N/A               | Torch Distribution object, containing
                                           | a batch of distributions describing
                                           | the policy for the provided observations.
            ``logp_a``   (batch,)          | Optional (only returned if batch of
                                           | actions is given). Tensor containing
                                           | the log probability, according to
                                           | the policy, of the provided actions.
                                           | If actions not given, will contain
                                           | ``None``.
            ===========  ================  ======================================

            The ``v`` module's forward call should accept a batch of observations
            and return:

            ===========  ================  ======================================
            Symbol       Shape             Description
            ===========  ================  ======================================
            ``v``        (batch,)          | Tensor containing the value estimates
                                           | for the provided observations. (Critical:
                                           | make sure to flatten this!)
            ===========  ================  ======================================


        ac_kwargs (dict): Any kwargs appropriate for the ActorCritic object
            you provided to PPO.

        seed (int): Seed for random number generators.

        steps_per_epoch (int): Number of steps of interaction (state-action pairs)
            for the agent and the environment in each epoch.

        epochs (int): Number of epochs of interaction (equivalent to
            number of policy updates) to perform.

        gamma (float): Discount factor. (Always between 0 and 1.)

        clip_ratio (float): Hyperparameter for clipping in the policy objective.
            Roughly: how far can the new policy go from the old policy while
            still profiting (improving the objective function)? The new policy
            can still go farther than the clip_ratio says, but it doesn't help
            on the objective anymore. (Usually small, 0.1 to 0.3.) Typically
            denoted by :math:`\epsilon`.

        pi_lr (float): Learning rate for policy optimizer.

        vf_lr (float): Learning rate for value function optimizer.

        train_pi_iters (int): Maximum number of gradient descent steps to take
            on policy loss per epoch. (Early stopping may cause optimizer
            to take fewer than this.)

        train_v_iters (int): Number of gradient descent steps to take on
            value function per epoch.

        lam (float): Lambda for GAE-Lambda. (Always between 0 and 1,
            close to 1.)

        max_ep_len (int): Maximum length of trajectory / episode / rollout.

        target_kl (float): Roughly what KL divergence we think is appropriate
            between new and old policies after an update. This will get used
            for early stopping. (Usually small, 0.01 or 0.05.)

        logger_kwargs (dict): Keyword args for EpochLogger.

        save_freq (int): How often (in terms of gap between epochs) to save
            the current policy and value function.

    """

    # Special function to avoid certain slowdowns from PyTorch + MPI combo.
    setup_pytorch_for_mpi()

    # Set up logger and save configuration
    logger = EpochLogger(**logger_kwargs)
    logger.save_config(locals())

    # Random seed
    seed += 10000 * proc_id()
    torch.manual_seed(seed)
    np.random.seed(seed)

    # Instantiate environment
    env = env_fn()
    obs_dim = env.observation_space.shape
    act_dim = env.action_space.shape


    print("obs dim " + str(obs_dim))
    print("act dim " + str(act_dim))

    # Create actor-critic module
    ac = actor_critic(env.observation_space, env.action_space, **ac_kwargs)

    # Sync params across processes
    sync_params(ac)

    # Count variables
    var_counts = tuple(core.count_vars(module) for module in [ac.pi, ac.v])
    logger.log('\nNumber of parameters: \t pi: %d, \t v: %d\n'%var_counts)

    # Set up experience buffer
    local_steps_per_epoch = int(steps_per_epoch / num_procs())
    buf = PPOBuffer(obs_dim, act_dim, local_steps_per_epoch, gamma, lam)

    # Set up function for computing PPO policy loss
    def compute_loss_pi(data):
        obs, act, adv, logp_old = data['obs'], data['act'], data['adv'], data['logp']

        # Policy loss
        pi, logp = ac.pi(obs, act)
        ratio = torch.exp(logp - logp_old)
        clip_adv = torch.clamp(ratio, 1-clip_ratio, 1+clip_ratio) * adv
        loss_pi = -(torch.min(ratio * adv, clip_adv)).mean()

        # Useful extra info
        approx_kl = (logp_old - logp).mean().item()
        ent = pi.entropy().mean().item()
        clipped = ratio.gt(1+clip_ratio) | ratio.lt(1-clip_ratio)
        clipfrac = torch.as_tensor(clipped, dtype=torch.float32).mean().item()
        pi_info = dict(kl=approx_kl, ent=ent, cf=clipfrac)

        return loss_pi, pi_info

    # Set up function for computing value loss
    def compute_loss_v(data):
        obs, ret = data['obs'], data['ret']
        return ((ac.v(obs) - ret)**2).mean()

    # Set up optimizers for policy and value function
    pi_optimizer = Adam(ac.pi.parameters(), lr=pi_lr)
    vf_optimizer = Adam(ac.v.parameters(), lr=vf_lr)

    # Set up model saving
    logger.setup_pytorch_saver(ac)

    def update():
        data = buf.get()

        pi_l_old, pi_info_old = compute_loss_pi(data)
        pi_l_old = pi_l_old.item()
        v_l_old = compute_loss_v(data).item()

        # Train policy with multiple steps of gradient descent
        for i in range(train_pi_iters):
            pi_optimizer.zero_grad()
            loss_pi, pi_info = compute_loss_pi(data)
            kl = mpi_avg(pi_info['kl'])
            if kl > 1.5 * target_kl:
                logger.log('Early stopping at step %d due to reaching max kl.'%i)
                break
            loss_pi.backward()
            mpi_avg_grads(ac.pi)    # average grads across MPI processes
            pi_optimizer.step()

        logger.store(StopIter=i)

        # Value function learning
        for i in range(train_v_iters):
            vf_optimizer.zero_grad()
            loss_v = compute_loss_v(data)
            loss_v.backward()
            mpi_avg_grads(ac.v)    # average grads across MPI processes
            vf_optimizer.step()

        # Log changes from update
        kl, ent, cf = pi_info['kl'], pi_info_old['ent'], pi_info['cf']
        logger.store(LossPi=pi_l_old, LossV=v_l_old,
                     KL=kl, Entropy=ent, ClipFrac=cf,
                     DeltaLossPi=(loss_pi.item() - pi_l_old),
                     DeltaLossV=(loss_v.item() - v_l_old))

    # Prepare for interaction with environment
    start_time = time.time()
    o, ep_ret, ep_len = env.reset(), 0, 0

    # Main loop: collect experience in env and update/log each epoch
    for epoch in range(epochs):
        for t in range(local_steps_per_epoch):
            a, v, logp = ac.step(torch.as_tensor(o, dtype=torch.float32))
            #print("🔥 PPO received state:", o.shape)

            next_o, r, d, _ = env.step(a)
            ep_ret += r
            ep_len += 1

            # save and log
            buf.store(o, a, r, v, logp)
            logger.store(VVals=v)

            # Update obs (critical!)
            o = next_o

            timeout = ep_len == max_ep_len
            terminal = d or timeout
            epoch_ended = t==local_steps_per_epoch-1

            if terminal or epoch_ended:
                if epoch_ended and not(terminal):
                    print('Warning: trajectory cut off by epoch at %d steps.'%ep_len, flush=True)
                # if trajectory didn't reach terminal state, bootstrap value target
                if timeout or epoch_ended:
                    _, v, _ = ac.step(torch.as_tensor(o, dtype=torch.float32))
                else:
                    v = 0
                buf.finish_path(v)
                if terminal:
                    # only save EpRet / EpLen if trajectory finished
                    logger.store(EpRet=ep_ret, EpLen=ep_len)
                o, ep_ret, ep_len = env.reset(), 0, 0


        # Save model
        if (epoch % save_freq == 0) or (epoch == epochs-1):
            logger.save_state({'env': env}, None)

        # Perform PPO update!
        update()

        # Log info about epoch
        logger.log_tabular('Epoch', epoch)
        logger.log_tabular('EpRet', with_min_and_max=True)
        logger.log_tabular('EpLen', average_only=True)
        logger.log_tabular('VVals', with_min_and_max=True)
        logger.log_tabular('TotalEnvInteracts', (epoch+1)*steps_per_epoch)
        logger.log_tabular('LossPi', average_only=True)
        logger.log_tabular('LossV', average_only=True)
        logger.log_tabular('DeltaLossPi', average_only=True)
        logger.log_tabular('DeltaLossV', average_only=True)
        logger.log_tabular('Entropy', average_only=True)
        logger.log_tabular('KL', average_only=True)
        logger.log_tabular('ClipFrac', average_only=True)
        logger.log_tabular('StopIter', average_only=True)
        logger.log_tabular('Time', time.time()-start_time)
        logger.dump_tabular()
    return ac


###################################################

In [32]:

import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--env', type=str, default=env_train) #'HalfCheetah-v2')


_StoreAction(option_strings=['--env'], dest='env', nargs=None, const=None, default=<stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x7c896088ce10>, type=<class 'str'>, choices=None, required=False, help=None, metavar=None)

In [33]:
#parser.add_argument('--hid', type=int, default=64)
#parser.add_argument('--l', type=int, default=2)
#parser.add_argument('--gamma', type=float, default=0.99)
#parser.add_argument('--seed', '-s', type=int, default=0)
#parser.add_argument('--cpu', type=int, default=4)
#parser.add_argument('--steps', type=int, default=4000)
#parser.add_argument('--epochs', type=int, default=50)
#parser.add_argument('--exp_name', type=str, default='ppo')



# Parser arguments adapted to hyperparameter optimization by chatGPT

parser.add_argument('--hid', type=int, default=512)  # Updated to match the first hidden size in ac_kwargs
parser.add_argument('--l', type=int, default=2)  # Updated to match the number of layers in ac_kwargs
parser.add_argument('--gamma', type=float, default=0.995)  # Updated to match gamma in ppo
parser.add_argument('--seed', '-s', type=int, default=42)  # Updated to match seed in ppo
parser.add_argument('--cpu', type=int, default=4)  # Kept as is since it's not in ppo
parser.add_argument('--steps', type=int, default=8000) #8000)  # Updated to match steps_per_epoch in ppo
parser.add_argument('--epochs', type=int, default=50) # 10 for test otherwise 100 it is too much 150)  # Updated to match epochs in ppo
parser.add_argument('--exp_name', type=str, default='ppo')  # Kept as is since it's not in ppo





_StoreAction(option_strings=['--exp_name'], dest='exp_name', nargs=None, const=None, default='ppo', type=<class 'str'>, choices=None, required=False, help=None, metavar=None)

In [34]:
#parser.add_argument('-f', '--file', type=str, help='Kernel connection file')  # Add this line
parser.add_argument('extra_args', nargs=argparse.REMAINDER)  # Catch-all for unrecognized arguments

# args = parser.parse_args()

#mpi_fork(args.cpu)  # run parallel code with mpi  doesn't work in colab

_StoreAction(option_strings=[], dest='extra_args', nargs='...', const=None, default=None, type=None, choices=None, required=True, help=None, metavar=None)

In [35]:
# Add this line to ignore unknown args
args, unknown = parser.parse_known_args()

In [36]:
import os
from google.colab import drive
drive.mount('/content/drive')

save_dir = "/content/drive/My Drive/FinRL/finrl_ppo_models"
os.makedirs(save_dir, exist_ok=True)

Mounted at /content/drive


In [37]:
save_dir = "/content/drive/My Drive/FinRL/finrl_ppo_models"
os.makedirs(save_dir, exist_ok=True)

In [38]:

from spinup.utils.run_utils import setup_logger_kwargs


logger_kwargs = setup_logger_kwargs(args.exp_name, args.seed)

trained_ppo=ppo(lambda : args.env, actor_critic=MLPActorCritic,
        ac_kwargs=dict(hidden_sizes=[args.hid]*args.l), gamma=args.gamma,
        seed=args.seed, steps_per_epoch=args.steps, epochs=args.epochs,
        logger_kwargs=logger_kwargs)



# Save the model
model_path = model_path = "/content/drive/My Drive/FinRL/finrl_ppo_models/finRL_working_subset_train_ppo_llm_50_epochs_8k_steps_01_.pth"
torch.save(trained_ppo.state_dict(), model_path)
print("Training finished and saved in " + model_path)
# Load the model
#loaded_ppo = MLPActorCritic()
#loaded_ppo.load_state_dict(torch.load(model_path))
#loaded_ppo.eval()  # Set the model to evaluation mode



#trained_ppo.save(TRAINED_MODEL_DIR + "/agent_ppo")


# ## Save the trained agent
# Trained agents should have already been saved in the "trained_models" drectory after you run the code blocks above.
#
# For Colab users, the zip files should be at "./trained_models" or "/content/trained_models".
#
# For users running on your local environment, the zip files should be at "./trained_models".

Logging data to /content/spinningup_pytorch/data/ppo/ppo_s42/progress.txt
Saving config:

{
    "ac_kwargs":	{
        "hidden_sizes":	[
            512,
            512
        ]
    },
    "actor_critic":	"MLPActorCritic",
    "clip_ratio":	0.7,
    "env_fn":	"<function <lambda> at 0x7c896075c4a0>",
    "epochs":	50,
    "exp_name":	"ppo",
    "gamma":	0.995,
    "lam":	0.95,
    "logger":	{
        "<spinup.utils.logx.EpochLogger object at 0x7c8968220ad0>":	{
            "epoch_dict":	{},
            "exp_name":	"ppo",
            "first_row":	true,
            "log_current_row":	{},
            "log_headers":	[],
            "output_dir":	"/content/spinningup_pytorch/data/ppo/ppo_s42",
            "output_file":	{
                "<_io.TextIOWrapper name='/content/spinningup_pytorch/data/ppo/ppo_s42/progress.txt' mode='w' encoding='utf-8'>":	{
                    "mode":	"w"
                }
            }
        }
    },
    "logger_kwargs":	{
        "exp_name":	"ppo",
        "